In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../resources')
from single_pad_transmon_pocket import TransmonPocket_Single

In [3]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [4]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [5]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

In [6]:
design.overwrite_enabled = True

options = dict()

q1 = TransmonPocket_Single(design,'Q1',options = dict(pad_width = '500um'))

  arr = construct_1d_object_array_from_listlike(values)



In [7]:
gui.rebuild()
gui.autoscale()

  arr = construct_1d_object_array_from_listlike(values)



In [8]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [9]:
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 15,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': True,
 'solution_order': 'High',
 'solver_type': 'Iterative'}

In [10]:
c1.sim.run(components=['Q1'])#, open_terminations=[('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])
c1.sim.capacitance_matrix #455um

INFO 01:09PM [connect_project]: Connecting to Ansys Desktop API...
INFO 01:09PM [load_ansys_project]: 	Opened Ansys App
INFO 01:09PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 01:09PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Berkelium/Documents/Ansoft/
	Project:   Project8
INFO 01:09PM [connect_design]: No active design found (or error getting active design).
INFO 01:09PM [connect]: 	 Connected to project "Project8". No design detected
INFO 01:09PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 01:09PM [connect_setup]: 	No design setup detected.
WARNING 01:09PM [connect_setup]: 	Creating Q3D default setup.
INFO 01:09PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:09PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 01:09PM [analyze]: Analyzing setup Setup
INFO 01:10PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Lo

,ground_main_plane,pad_top_Q1
ground_main_plane,159.70849,-114.59787
pad_top_Q1,-114.59787,118.14222


In [11]:
c1.sim._get_results_from_renderer()
c_mat = c1.sim.capacitance_matrix

INFO 01:11PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmptwqx14g_.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 01:11PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmps30ju9c_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 01:11PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmpr4hdbjog.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 01:11PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmpv7agthie.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 01:11PM [get_matrix]: Exporting matrix data to (C:\Users\BERKEL~1\AppData\Local\Temp\tmph908ve5j.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 4, F

In [18]:
cap = -c_mat['ground_main_plane']['pad_top_Q1']

In [ ]:
Ejs = []

In [4]:
import numpy as np
import scipy.linalg as la
import qutip as q
import astropy.units as u
import astropy.constants as con

In [6]:
## Default constant
eq = 1.602e-19*u.C 
epsilon = con.eps0
h = 6.636/10**34
hbar = con.hbar
kb = con.k_B
Tc = 1.2
delta = 1.764*kb*Tc
phi0 = con.h/2/eq
dielectric = 9.34*epsilon

In [8]:
u.fF

Unit("fF")

In [ ]:
import numpy as np
import scipy.linalg as la
import qutip as q
import astropy.units as u
import astropy.constants as con

class Single_Transmon_Circuit:
    ## Default constant
    eq = 1.602e-19*u.C 
    epsilon = con.eps0
    h = 6.636/10**34
    hbar = con.hbar
    kb = con.k_B
    Tc = 1.2
    delta = 1.764*kb*Tc
    phi0 = con.h/2/eq
    dielectric = 9.34*epsilon
    def __init__(self,cap_mat,pad_name,CPW_name,ground_plane_name = 'ground_main_plane') -> None:
        self.c_couple = -cap_mat[pad_name][CPW_name]*u.fF
        self.c_to_ground = cap_mat[pad_name][pad_name]
        self.c_CPW = cap_mat[CPW_name][CPW_name]